In [ ]:
pip install igdb-api-v4

In [ ]:
from igdb.wrapper import IGDBWrapper
import json
import pandas as pd
import time

In [ ]:
with open('igdb_credentials.json', 'r') as f:
    creds = json.load(f)

CLIENT_ID = creds['client_id']
ACCESS_TOKEN = creds['access_token']

In [ ]:
wrapper = IGDBWrapper(CLIENT_ID, ACCESS_TOKEN)

In [ ]:
all_games = []
offset = 0
limit = 500  # IGDB API only can get 500 row at one time

while True:
    query = f'''
    fields id, name, genres, game_type, first_release_date, aggregated_rating, aggregated_rating_count, rating, rating_count, total_rating, total_rating_count;
    where platforms = (6);
    limit {limit};
    offset {offset};
    '''

    byte_array = wrapper.api_request('games', query)
    games_batch = json.loads(byte_array.decode('utf-8'))

    if not games_batch:
        break

    all_games.extend(games_batch)
    offset += limit
    print(f'Fetched {len(games_batch)} games (total so far: {len(all_games)})')
    time.sleep(0.25)

print(f'Done! Total games fetched: {len(all_games)}')

Fetched 500 games (total so far: 500)
Fetched 500 games (total so far: 1000)
Fetched 500 games (total so far: 1500)
Fetched 500 games (total so far: 2000)
Fetched 500 games (total so far: 2500)
Fetched 500 games (total so far: 3000)
Fetched 500 games (total so far: 3500)
Fetched 500 games (total so far: 4000)
Fetched 500 games (total so far: 4500)
Fetched 500 games (total so far: 5000)
Fetched 500 games (total so far: 5500)
Fetched 500 games (total so far: 6000)
Fetched 500 games (total so far: 6500)
Fetched 500 games (total so far: 7000)
Fetched 500 games (total so far: 7500)
Fetched 500 games (total so far: 8000)
Fetched 500 games (total so far: 8500)
Fetched 500 games (total so far: 9000)
Fetched 500 games (total so far: 9500)
Fetched 500 games (total so far: 10000)
Fetched 500 games (total so far: 10500)
Fetched 500 games (total so far: 11000)
Fetched 500 games (total so far: 11500)
Fetched 500 games (total so far: 12000)
Fetched 500 games (total so far: 12500)
Fetched 500 games (t

In [ ]:
game_ids = [g['id'] for g in all_games]

#popularity_primitives endpoint
def get_popularity_data(game_ids_batch):
    id_list = ','.join(map(str, game_ids_batch))
    query = f'''
    fields game_id, value, external_popularity_source, popularity_type;
    where game_id = ({id_list});
    '''
    byte_array = wrapper.api_request('popularity_primitives', query)
    return json.loads(byte_array.decode('utf-8'))

popularity_records = []
batch_size = 200
for i in range(0, len(game_ids), batch_size):
    batch = game_ids[i:i+batch_size]
    popularity_records.extend(get_popularity_data(batch))
    time.sleep(0.2)

# Convert dict to game_id
popularity_dict = {r['game_id']: r for r in popularity_records}

# Merged to all_games
for game in all_games:
    gid = game['id']
    record = popularity_dict.get(gid, {})
    game['popularity_score'] = record.get('value')
    game['popularity_source_id'] = record.get('external_popularity_source')
    game['popularity_type_id'] = record.get('popularity_type')


In [ ]:
#To reduce loading time for each cell, remove games that do not have a popularity score available.
all_games = [g for g in all_games if g.get('popularity_score') is not None]
print(f"Total games in list: {len(all_games)}")

Total games in list: 2277


In [ ]:
#get information related to game data time
game_ids = [g['id'] for g in all_games]

def get_time_to_beat_data(game_ids_batch):
    id_list = ','.join(map(str, game_ids_batch))
    query = f'''
    fields game_id, normally, hastily, completely, count;
    where game_id = ({id_list});
    '''
    byte_array = wrapper.api_request('game_time_to_beats', query)
    return json.loads(byte_array.decode('utf-8'))


time_to_beat_records = []
batch_size = 200
for i in range(0, len(game_ids), batch_size):
    batch = game_ids[i:i+batch_size]
    time_to_beat_records.extend(get_time_to_beat_data(batch))
    time.sleep(0.2)


ttb_dict = {r['game_id']: r for r in time_to_beat_records}

# merge the gametime back to all games list
for game in all_games:
    ttb = ttb_dict.get(game['id'], {})
    game['ttb_normally'] = ttb.get('normally')
    game['ttb_hastily'] = ttb.get('hastily')
    game['ttb_completely'] = ttb.get('completely')
    game['ttb_submission_count'] = ttb.get('count')


In [ ]:
# Get Genre list
query = '''
fields id, name;
limit 500;
'''
byte_array = wrapper.api_request('genres', query)
genre_list = json.loads(byte_array.decode('utf-8'))

# genre to game id
genre_dict = {g['id']: g['name'] for g in genre_list}

# Convert genre ID list to name list
for game in all_games:
    genre_ids = game.get('genres')
    if not isinstance(genre_ids, list):
        genre_ids = []
    game['genre_names'] = [genre_dict.get(i, 'unknown') for i in genre_ids]

In [ ]:
df = pd.DataFrame(all_games)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2277 non-null   int64  
 1   aggregated_rating        1060 non-null   float64
 2   aggregated_rating_count  1060 non-null   float64
 3   first_release_date       2248 non-null   float64
 4   genres                   2243 non-null   object 
 5   name                     2277 non-null   object 
 6   rating                   1546 non-null   float64
 7   rating_count             1546 non-null   float64
 8   total_rating             1718 non-null   float64
 9   total_rating_count       1718 non-null   float64
 10  game_type                2277 non-null   int64  
 11  popularity_score         2277 non-null   float64
 12  popularity_source_id     2277 non-null   int64  
 13  popularity_type_id       2277 non-null   int64  
 14  ttb_normally            

In [ ]:
df.to_csv("pc_games_igdb .csv", index=False, encoding='utf-8')